In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 7.6MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 7.4MB/s 
     |████████████████████████████████| 901kB 45.8MB/s 
     |████████████████████████████████| 3.3MB 41.6MB/s 


In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
import pandas as pd
text = pd.read_csv("stockholm/xlm_code/mohx_xlm_subs/mohx_tp.csv")

In [ ]:
text.shape

(31, 5)

In [ ]:
text.to_csv(r'mohx_xlm_tp.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
with open('stockholm/mlm/preds/mohx_xlm_tp.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base').eval()

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').eval()

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

In [ ]:
text

['" A hot soup will revive me ." soup revive 1 1',
 '" He revived this style of opera ." style revive 1 1',
 '" She designed a good excuse for not attending classes that day ." excuse design 1 1',
 '" The colors do n\'t harmonize ." color harmonize 1 1',
 '" society was infected by racism ." society infect 1 1',
 '" The market is being clogged by these operations ." market clog 1 1',
 '" The mother was swelling with importance when she spoke of her son ." mother swell 1 1',
 '" Their language inclines us to believe them ." language incline 1 1',
 '" capture the essence of Spring   ." essence capture 1 1',
 '" The scientists had to accommodate the new results with the existing theories ." result accommodate 1 1',
 '" She was besieged by so many problems that she got discouraged ." problem besiege 1 1',
 '" They fastened various nicknames to each other ." nickname fasten 1 1',
 '" He showered her with presents ." present shower 1 1',
 '" Am I supposed to swallow that story ?" story swall

In [ ]:
text_sentence = 'A hot <mask> will revive me .'
get_all_predictions(text_sentence)

A hot <mask> will revive me .


{'bart': 'shower\ncup\nbath\ntodd\ndrink',
 'bert': 'shower\nbath\nmeal\nwater\ndrink',
 'electra': 'summer\nsun\nspring\nday\ncold',
 'roberta': 'bath\nshower\ndrink\ntea\niron',
 'xlm': 'girl\nshower\ndog\nsummer\nday',
 'xlnet': 'will\nhot\nlead\nstar\nhit'}

In [ ]:
text_sentence = " He revived this <mask> of opera ."
get_all_predictions(text_sentence)

 He revived this <mask> of opera .


{'bart': 'classic\ngreat\ntradition\ngenre\nform',
 'bert': 'form\nstyle\ngenre\ntype\nkind',
 'electra': 'style\nform\nkind\ntype\ntradition',
 'roberta': 'form\nkind\ngenre\ntype\nstyle',
 'xlm': 'piece\nstyle\nopera\ntype\nsort',
 'xlnet': 'this\nrevival\ntradition\nform\ntype'}

In [ ]:
text_sentence = " They clawed their <mask> to the top of the mountain ."
get_all_predictions(text_sentence)

 They clawed their <mask> to the top of the mountain .


{'bart': 'way\nown\nfeet\ntheir\nhands',
 'bert': 'way\nways\nfeet\nnails\nhands',
 'electra': 'way\nways\nhands\npath\nclaws',
 'roberta': 'way\nfeet\nfingers\nways\nclimb',
 'xlm': 'feet\nway\narms\nhead\nback',
 'xlnet': 'way\nup\nupward\nback\nit'}

In [ ]:
text_sentence = "We rotate the <mask> so as to maximize the use of the soil ."
get_all_predictions(text_sentence)

We rotate the <mask> so as to maximize the use of the soil .


{'bart': 'soil\nplant\nroot\nwheel\nplants',
 'bert': 'soil\nearth\nstones\nfields\nplants',
 'electra': 'water\nplants\nplates\nsystem\nbases',
 'roberta': 'soil\ncompost\ncrop\nplant\nplants',
 'xlm': 'plant\ngarden\ntree\nsurface\nfield',
 'xlnet': 'so\nin\nthe\nabout\nto'}

In [ ]:
text_sentence = "She designed a good <mask> for not attending classes that day ."
get_all_predictions(text_sentence)

She designed a good <mask> for not attending classes that day .


{'bart': 'excuse\nreason\nargument\nexplanation\none',
 'bert': 'excuse\nreason\ndress\noutfit\ndisguise',
 'electra': 'excuse\nplan\nsystem\ndiet\nenvironment',
 'roberta': 'excuse\ncase\nstrategy\nreason\nplan',
 'xlm': 'excuse\nreason\nexplanation\nargument\ncase',
 'xlnet': 'good\ndeal\nbad\nGood\nhead'}

In [ ]:
text_sentence = "The colors do n\'t <mask> ."
get_all_predictions(text_sentence)

The colors do n't <mask> .


{'bart': 'match\nchange\nlook\nappear\nhave',
 'bert': 'do\nwork\ngo\nchange\nfade',
 'electra': 'matter\nchange\nshow\nwork\nmatch',
 'roberta': 'change\nmatter\nmatch\nmix\nwork',
 'xlm': 'matter\nmatch\nwork\nchange\nfit',
 'xlnet': 'not\ndo\nand\nhave\nbut'}

In [ ]:
text_sentence = "society was <mask> by racism ."
get_all_predictions(text_sentence)

society was <mask> by racism .


{'bart': 'not\nstill\ndivided\nalso\ndominated',
 'bert': 'characterized\ndominated\nmarked\ncharacterised\ndefined',
 'electra': 'plagued\ndriven\ndominated\nravaged\naffected',
 'roberta': 'shaped\ndefined\ndestroyed\ndominated\nplagued',
 'xlm': 'created\naffected\nrun\nmade\nchanged',
 'xlnet': 'was\nby\nwere\nin\nis'}

In [ ]:
text_sentence = "The market is being <mask> by these operations ."
get_all_predictions(text_sentence)

The market is being <mask> by these operations .


{'bart': 'served\nfurther\ndominated\ndriven\nsupported',
 'bert': 'served\ndominated\ndeveloped\nserviced\naffected',
 'electra': 'fed\ndriven\naffected\ndominated\ncontrolled',
 'roberta': 'served\ndominated\npenetrated\ndriven\ndisrupted',
 'xlm': 'affected\nfilled\nreached\ncreated\nused',
 'xlnet': 'being\ngoing\ning\ntaking\nwas'}

In [ ]:
text_sentence = "The mother was <mask> with importance when she spoke of her son ."
get_all_predictions(text_sentence)

The mother was <mask> with importance when she spoke of her son .


{'bart': 'visibly\nalways\nalso\nnot\nsaid',
 'bert': 'filled\nspeaking\nstruck\nspoken\nheavy',
 'electra': 'filled\ntreated\nmet\npresented\ncharged',
 'roberta': 'speaking\nfilled\nglowing\nstruck\noverwhelmed',
 'xlm': 'treated\ntaken\nfilled\nimpressed\nconcerned',
 'xlnet': 'was\nof\nvery\nin\ngreat'}

In [ ]:
text_sentence = "Their language <mask> us to believe them ."
get_all_predictions(text_sentence)

Their language <mask> us to believe them .


{'bart': 'is\nand\nmay\nhas\ntells',
 'bert': 'led\nleads\nconvinced\nforced\nencouraged',
 'electra': 'allows\ntells\nwants\nallowed\nrequires',
 'roberta': 'begs\nwants\ntells\nforces\ninvites',
 'xlm': 'allows\ntaught\nforces\ncauses\nwants',
 'xlnet': 'language\nis\nlanguages\nyou\nus'}

In [ ]:
text_sentence = "capture the <mask> of Spring   ."
get_all_predictions(text_sentence)

capture the <mask> of Spring   .


{'bart': 'beauty\nspirit\nmoment\nmagic\nessence',
 'bert': 'light\nheat\nbreath\nrite\nscent',
 'electra': 'image\nlight\nscent\nblood\nshadow',
 'roberta': 'spirit\nbeauty\npromise\nRite\nSpirit',
 'xlm': 'beauty\nsky\nseason\ncolors\nflowers',
 'xlnet': 'of\nthe\nspirit\nday\nculture'}

In [ ]:
text_sentence = "The scientists had to accomodate the new <mask> with the existing theories ."
get_all_predictions(text_sentence)

The scientists had to accomodate the new <mask> with the existing theories .


{'bart': 'theory\nfindings\ntheories\nideas\ndata',
 'bert': 'scientists\ntheories\nexperiments\nresearchers\nscientist',
 'electra': 'theories\ntheory\nhypothesis\nideas\nfindings',
 'roberta': 'theories\ntheory\nfindings\nresults\ndata',
 'xlm': 'theory\nmodels\nmethods\nresearch\ndata',
 'xlnet': 'new\nthing\nalong\ndeal\naddition'}

In [ ]:
text_sentence = "She was besieged by so many <mask> that she got discouraged ."
get_all_predictions(text_sentence)

She was besieged by so many <mask> that she got discouraged .


{'bart': 'people\nmen\nof\nenemies\nother',
 'bert': 'men\npeople\nthings\nquestions\nproblems',
 'electra': 'people\nenemies\nmen\ndemons\nchildren',
 'roberta': 'men\nenemies\nsoldiers\npeople\nthreats',
 'xlm': 'things\npeople\nwomen\nproblems\nwords',
 'xlnet': 'things\nmuch\ntimes\npeople\nways'}

In [ ]:
text_sentence = "They fastened various <mask> to each other ."
get_all_predictions(text_sentence)

They fastened various <mask> to each other .


{'bart': 'items\npieces\nobjects\nrings\nkinds',
 'bert': 'objects\nitems\nthings\ndevices\ninstruments',
 'electra': 'ties\nstraps\nlayers\nbuttons\nlinks',
 'roberta': 'items\nchains\nobjects\nthings\nbonds',
 'xlm': 'things\nwords\nparts\nletters\npoints',
 'xlnet': 'together\non\nrelative\nthings\nvarious'}

In [ ]:
text_sentence = "He showered her with <mask> ."
get_all_predictions(text_sentence)

He showered her with <mask> .


{'bart': 'kisses\ngifts\naffection\na\nlove',
 'bert': 'kisses\nsoap\nwarmth\nwater\nheat',
 'electra': 'water\nkisses\nflowers\ntea\ntears',
 'roberta': 'gifts\nflowers\nkisses\ncompliments\npraise',
 'xlm': 'sperm\nwater\ncum\nmilk\nwet',
 'xlnet': 'a\nwith\nby\nand\nthis'}

In [ ]:
text_sentence = "Am I supposed to swallow that <mask> ?"
get_all_predictions(text_sentence)

Am I supposed to swallow that <mask> ?


{'bart': 'pill\nwhole\nshit\nand\none',
 'bert': 'pill\none\nshit\ndown\nthing',
 'electra': 'pill\nshit\nthing\nnow\none',
 'roberta': 'pill\nshit\ncrap\none\ncake',
 'xlm': 'liquid\nwater\npill\nthing\nmilk',
 'xlnet': 'that\nthis\na\nwhich\nthen'}

In [ ]:
text_sentence = "My skin <mask> -- I was terrified ."
get_all_predictions(text_sentence)

My skin <mask> -- I was terrified .


{'bart': 'was\ncrawled\nfelt\nturned\nprick',
 'bert': 'crawled\nburned\nhurt\nchilled\nfelt',
 'electra': 'was\ncolor\nfelt\nis\ncolour',
 'roberta': 'crawled\nburned\nfroze\ndarkened\ntightened',
 'xlm': 'color\ntone\ncare\ncare\nrose',
 'xlnet': 'skin\ns\nture\nI\nfear'}

In [ ]:
text_sentence = "My skin <mask> -- I was terrified ."
get_all_predictions(text_sentence)

In [ ]:
text_sentence = "We poured <mask> into the education of our children ."
get_all_predictions(text_sentence)

We poured <mask> into the education of our children .


{'bart': 'a\nmillions\nover\nour\nbillions',
 'bert': 'money\nourselves\nenergy\nresources\nit',
 'electra': 'money\nourselves\nenergy\nwater\nmore',
 'roberta': 'ourselves\nresources\nmoney\nmillions\nbillions',
 'xlm': 'passion\nfaith\nourselves\nconfidence\nwell',
 'xlnet': 'into\nover\nin\nout\npoured'}

In [ ]:
text_sentence = "The sun <mask> down on us ."
get_all_predictions(text_sentence)

The sun <mask> down on us .


{'bart': 'is\nwill\nwas\nhas\ncomes',
 'bert': 'beat\nshone\nbeats\ncame\nbore',
 'electra': 'came\nwas\nwent\nshone\nis',
 'roberta': 'beat\ncame\nbeats\nwent\nshone',
 'xlm': 'came\nwent\nis\nwas\nlooked',
 'xlnet': 'down\nsun\nup\nback\nbody'}

In [ ]:
text_sentence = "Unrest <mask> in the country ."
get_all_predictions(text_sentence)

Unrest <mask> in the country .


{'bart': 'continues\nis\nhas\nand\nin',
 'bert': 'erupted\ncontinued\noccurred\nbegan\nspread',
 'electra': 'continues\ncontinued\nremains\noccurred\nexists',
 'roberta': 's\ncontinues\nrising\npersists\ngrowing',
 'xlm': 'ling\nled\nlement\ns\nration',
 'xlnet': 'in\nhere\nis\nIn\nthere'}

In [ ]:
text_sentence = "poison someone \'s <mask> ."
get_all_predictions(text_sentence)

poison someone 's <mask> .


{'bart': 'water\ndrink\nbrain\nmouth\nblood',
 'bert': 'blood\nbody\nheart\nmind\nlife',
 'electra': 'body\nmouth\nmind\nskin\nface',
 'roberta': 'life\nname\nhead\nblood\nsoul',
 'xlm': 'life\nmind\nbody\nface\nhead',
 'xlnet': 's\nhouse\nplace\nhome\nbusiness'}

In [ ]:
text_sentence = "Sales were <mask> after prices were lowered ."
get_all_predictions(text_sentence)

Sales were <mask> after prices were lowered .


{'bart': 'also\ndown\nup\nslightly\nflat',
 'bert': 'increased\nreduced\nboosted\nimproved\ndiscontinued',
 'electra': 'halted\nreduced\nclosed\nsuspended\nstopped',
 'roberta': 'steady\nup\nhigher\nflat\nstable',
 'xlm': 'increased\nclosed\nhigher\nstrong\nlower',
 'xlnet': 'until\nwere\nagain\ntoday\nwhere'}

In [ ]:
text_sentence = "This behavior will ruin your <mask> of winning the election ."
get_all_predictions(text_sentence)

This behavior will ruin your <mask> of winning the election .


{'bart': 'chances\nchance\ncampaign\nparty\nhopes',
 'bert': 'chances\nchance\nprospects\nhope\nhopes',
 'electra': 'chances\nchance\nhopes\nhope\nrecord',
 'roberta': 'chances\nchance\nhopes\nprospects\nodds',
 'xlm': 'chances\nchance\nodds\ndream\nprospect',
 'xlnet': 'of\noff\nnot\nif\nfailing'}

In [ ]:
text_sentence = "The sales tax is <mask> into the state income tax ."
get_all_predictions(text_sentence)

The sales tax is <mask> into the state income tax .


{'bart': 'not\nalso\nthen\ncollected\nincorporated',
 'bert': 'incorporated\nconverted\nintegrated\nmerged\ncombined',
 'electra': 'converted\ndivided\nincorporated\nturned\ntranslated',
 'roberta': 'absorbed\nfolded\nrolled\nincorporated\nmerged',
 'xlm': 'included\nadded\npaid\nbuilt\nmade',
 'xlnet': 'into\nin\nout\nover\ngoing'}

In [ ]:
text_sentence = "The invaders <mask> their language all over the country ."
get_all_predictions(text_sentence)

The invaders <mask> their language all over the country .


{'bart': 'spread\nused\nleft\nwere\nhave',
 'bert': 'spread\nspoke\nused\nchanged\ntook',
 'electra': 'spread\nspeak\nspoken\nspoke\nread',
 'roberta': 'spread\nused\npracticed\nspoke\nintroduced',
 'xlm': 'spoke\nused\nspeak\nspread\nuse',
 'xlnet': 'have\nput\nof\nare\ntook'}

In [ ]:
text_sentence = "Interns have to <mask> for a few months ."
get_all_predictions(text_sentence)

Interns have to <mask> for a few months .


{'bart': 'stay\nbe\nlive\nwork\nattend',
 'bert': 'stay\nwork\nwait\nleave\nlive',
 'electra': 'stay\nwait\nleave\nwork\nsleep',
 'roberta': 'stay\nleave\nwait\nwork\nreturn',
 'xlm': 'work\nwait\nstay\nleave\napply',
 'xlnet': 'go\nbe\nstay\nfor\nwait'}

In [ ]:
text_sentence = "They sifted through the job <mask> ."
get_all_predictions(text_sentence)

They sifted through the job <mask> .


{'bart': 'applications\ndescriptions\ndescription\napplication\nfiles',
 'bert': 'files\ndescription\ndescriptions\nfile\nlist',
 'electra': 'again\npapers\nwork\nnow\ndone',
 'roberta': 'listings\ndescriptions\napplications\nmarket\noffers',
 'xlm': 'and\non\nof\nwith\ndone',
 'xlnet': 'body\nmarket\nposition\nthat\npath'}

In [ ]:
text_sentence = "The dress <mask> her beautiful figure ."
get_all_predictions(text_sentence)

The dress <mask> her beautiful figure .


{'bart': 'accent\nalso\nshowed\nperfectly\ncomple',
 'bert': 'hugged\nrevealed\nframed\nshowcased\noutlined',
 'electra': 'is\nwas\nfeatures\nshows\nfeatured',
 'roberta': 'hugged\nhighlighted\nemphasized\nhighlights\nreflected',
 'xlm': 'shows\nshowed\nof\nand\nmakes',
 'xlnet': 'her\nhis\nmy\nshe\nhe'}

In [ ]:
text_sentence = "Fill the <mask> with pride ."
get_all_predictions(text_sentence)

Fill the <mask> with pride .


{'bart': 'home\nworld\nhouse\nroom\nhalls',
 'bert': 'room\nworld\nair\nhouse\ncrowd',
 'electra': 'world\nroom\nspace\neyes\nwords',
 'roberta': 'air\nroom\nhouse\nhalls\nvoid',
 'xlm': 'table\nday\nroom\npage\nface',
 'xlnet': 'the\nwith\nin\nup\nfull'}

In [ ]:
text_sentence = "The listeners <mask> when he discussed his strange ideas ."
get_all_predictions(text_sentence)

The listeners <mask> when he discussed his strange ideas .


{'bart': 'were\nlaughed\nalso\nloved\nhad',
 'bert': 'laughed\nlistened\nresponded\ngiggled\nsmiled',
 'electra': 'laughed\nagreed\ngasped\ncheered\nsmiled',
 'roberta': 'laughed\ncheered\ngasped\nlistened\napplauded',
 'xlm': 'laugh\nsurprised\nwatched\nenjoyed\nsmile',
 'xlnet': 'miss\near\nlisteners\nthat\nears'}

In [ ]:
text_sentence = "The cancer had invaded her <mask> ."
get_all_predictions(text_sentence)

The cancer had invaded her <mask> .


{'bart': 'brain\nliver\nlungs\nbones\nbody',
 'bert': 'brain\nbody\nskin\nmind\nsystem',
 'electra': 'body\nlife\nheart\nhome\nbrain',
 'roberta': 'brain\nlungs\nbody\ncolon\nliver',
 'xlm': 'body\nlife\nbrain\nskin\nhome',
 'xlnet': 'today\nto\nand\na\np'}

In [ ]:
text_sentence = "The path <mask> all the way to the top of the hill ."
get_all_predictions(text_sentence)

The path <mask> all the way to the top of the hill .


{'bart': 'is\ncontinues\nfollows\nruns\nleads',
 'bert': 'led\nwent\nran\ncontinued\nstretched',
 'electra': 'goes\nruns\ncontinues\nleads\nis',
 'roberta': 'goes\ncontinues\nruns\nleads\nextends',
 'xlm': 'is\ngoes\nruns\ncontinues\ns',
 'xlnet': 'all\nup\nback\ngoing\npath'}

In [ ]:
text_sentence = "condense the <mask> of a book into a summary ."
get_all_predictions(text_sentence)

condense the <mask> of a book into a summary .


{'bart': 'contents\nmain\nfirst\nmost\nentire',
 'bert': 'text\ncontents\npages\nwords\ncontent',
 'electra': 'contents\nrest\ntitle\ndetails\nremainder',
 'roberta': 'contents\ncontent\nwhole\ntext\nentirety',
 'xlm': 'content\ntitle\ntext\npages\nbeginning',
 'xlnet': 'out\nof\nway\nall\nlength'}

In [ ]:
text_sentence = "This poses an interesting <mask> ."
get_all_predictions(text_sentence)

This poses an interesting <mask> .


{'bart': 'question\nchallenge\nproblem\ndilemma\ncon',
 'bert': 'problem\nchallenge\nquestion\nissue\npossibility',
 'electra': 'challenge\nsituation\nissue\nmoment\npicture',
 'roberta': 'question\nproblem\nchallenge\nissue\ndilemma',
 'xlm': 'problem\nquestion\ndilemma\nchallenge\nvraag',
 'xlnet': 'a\nin\nfirst\nto\nor'}